# Libraries & Dates

## Libraries

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Authenticate and grant permissions to connect with Google Sheets
from google.colab import auth
auth.authenticate_user()

# Import necessary libraries for Google Sheets integration
from gspread_dataframe import set_with_dataframe
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Libraries for data manipulation and analysis
import pandas as pd
import numpy as np

# Module to find/search files in a directory according to a pattern
import glob

# Library to handle regular expressions for removing decimal points
import re

import time

from datetime import datetime, timedelta

Mounted at /content/drive


## Static Variables & funtions

## Dates

In [ ]:
current_date = datetime.now().date()

# Dataframes filter
filter_start_date = current_date - timedelta(days=180)
filter_start_date = filter_start_date.strftime("%Y-%m-%d")

# ReContacto Auditable base
start_date = '2025-02-01'
end_date = '2025-02-28'
file_name = '/content/drive/MyDrive/Base Auditable - Febrero 2025.csv'

# Cami´s Info (CR MKP)
mkp_date_start = '2024-01-01'

# RRSS P.
hojas_rangos = [
#('ENERO 2024','B5:G3000'),('FEBRERO 2024','B5:G3000'),('MARZO 2024','B5:G3000'),('ABRIL 2024','B5:G3000'),('MAYO 2024','B5:G3000'),('JUNIO 2024','B5:G3000')
#,('JULIO 2024','B5:G3000'),('AGOSTO 2024','B5:G3000'),('SEPTIEMBRE 2024','B5:G3000'),('OCTUBRE 2024','B5:G3000')
    ('NOVIEMBRE 2024' , 'B5:G3000')
    ,('DICIEMBRE 2024'  , 'B5:G3000')
    ,('ENERO 2025'  , 'B5:G3000')
    ,('FEBRERO 2025'  , 'B5:G3000')
]

# Caidas
sheet_names = [
#'OCTUBRE 2024'
    'NOVIEMBRE 2024'
    ,'DICIEMBRE 2024'
    ,'ENERO 2025'
    ,'FEBRERO 2025'

]

# Extraction

## Easiers

In [ ]:
### MKP info | Mirakl Download
mkp = pd.read_csv(mirakl_download, sep=';').fillna('')

<ipython-input-100-a10f6d7ef8c9>:2: DtypeWarning: Columns (32,43,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  mkp = pd.read_csv(mirakl_download, sep=';').fillna('')


In [ ]:
### BP Sales | RPA mainBP.py
for file in glob.glob(f'{rpa_bp_sales}/*.csv'):
    dfS = pd.read_csv(file, sep=',')
    dfS = dfS.dropna(thresh=2)
    dfS = dfS.fillna('')
    dataframes_bp_sales.append(dfS)
    sales_bp = pd.concat(dataframes_bp_sales, ignore_index=True)

"\n### SALES by Revenue\n\ngsheetsales = gc.open_by_key('1NLUh-r2OFyWrwBSX6S2UKUQFo4DeyhX6OKYier9nF_w').worksheet('SALES')\n\nsales_bp = pd.DataFrame(gsheetsales.get_all_values()[1:], columns=gsheetsales.get_all_values()[0])\n"

In [ ]:
### Incidents | csv on drive
novelties = pd.concat([pd.read_csv(carga) for carga in glob.glob(incidents_files)], ignore_index=True)

In [ ]:
### "Llamadas Caidas"

#### Function to fetch data from Google Sheets
def fetch_data(sheet_key, sheet_names):
    try:
        #### Open the Google Sheet by key
        spreadsheet = gc.open_by_key(sheet_key)
        available_sheets = [sheet.title for sheet in spreadsheet.worksheets()]
        collected_data = []

        #### Iterate over the desired sheet names
        for sheet_name in sheet_names:
            if sheet_name in available_sheets:
                sheet = spreadsheet.worksheet(sheet_name)
                data = sheet.get_all_values()
                if data:
                    dfc = pd.DataFrame(data[1:], columns=data[0])
                    collected_data.append(dfc)

        #### Combine data from all sheets into a single DataFrame
        if collected_data:  # Check if there is any data to combine
            final_df = pd.concat(collected_data, ignore_index=True)
            return final_df
        else:
            print("No data found in the specified sheets.")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error processing sheet {sheet_key}: {e}")
        return pd.DataFrame()

#### Fetch data for the given sheet key and sheet names
final_dataframe = fetch_data(sheet_key, sheet_names)

In [ ]:
### HeadCount
gsheet1 = gc.open_by_key(hc).worksheet('HC actualizado')

hc_data = pd.DataFrame(gsheet1.get_all_values()[1:], columns=gsheet1.get_all_values()[0])

In [ ]:
### Matrix Motivos de consulta
gsheetcatandmot = gc.open_by_key(gheet_matrix).worksheet('Matrix')

catmatrix = pd.DataFrame(gsheetcatandmot.get_all_values()[1:], columns=gsheetcatandmot.get_all_values()[0])

In [ ]:
### Phone IQ
iq = pd.concat([pd.read_csv(archivo) for archivo in glob.glob(f'{phoneiq_download}/*.csv')], ignore_index=True).drop_duplicates(subset=['uniqueid']).fillna('')

In [ ]:
### Zendesk Incoming
vacio = pd.concat([pd.read_csv(archivo, sep=';') for archivo in glob.glob(f'{zendesk_download}/*.csv')], ignore_index=True)

In [ ]:
### Zendesk - Productivity
products = pd.DataFrame()

for carpeta, canal in drives_productivity.items():
    csv_files = glob.glob(carpeta)
    for archivo in csv_files:
        df_archivo = pd.read_csv(archivo, sep=';')
        df_archivo['Channel'] = canal
        products = pd.concat([products, df_archivo], ignore_index=True)

In [ ]:
### Zendesk - Productivity | %No Solved
chat_solved = pd.concat([pd.read_csv(carga,sep=';') for carga in glob.glob(no_solved)], ignore_index=True)

## Harder

In [ ]:
# Introduce a delay to respect Google Sheets API rate limits
time.sleep(60) # Wait for 60 seconds

### Public RRSS
def obtener_datos_agente(archivo_key, agente_nombre):
    gestiones = []
    archivo = gc.open_by_key(archivo_key)
    hojas_disponibles = [hoja.title for hoja in archivo.worksheets()]

    for nombre_hoja, rango in hojas_rangos:
        if nombre_hoja in hojas_disponibles:
            hoja = archivo.worksheet(nombre_hoja)
            datos_hoja = hoja.batch_get([rango])
            gestiones.append(pd.DataFrame(datos_hoja[0][1:], columns=columnas_interes))
    dfg = pd.concat(gestiones, ignore_index=True)
    dfg['Nombre del agente asignado'] = agente_nombre
    return dfg

agentes_df = [obtener_datos_agente(archivo_key, agente_nombre) for archivo_key, agente_nombre in archivos.items()]
allagents = pd.concat(agentes_df, ignore_index=True)

# Transformation

In [ ]:
#################################
### MKP INFO
mkp['Date created'] = pd.to_datetime(mkp['Date created'])
mkp['Order ID'] = mkp['Order number'].str.split('-').str[0]
mkp['Country MKP'] = mkp['Order ID'].apply(determinar_pais)

#### Dataframe 2
mkp_filtrado = mkp.groupby(['Order ID', 'Shipping address last name', 'Shipping address country', 'Country MKP','Date created']).agg({
    'Store': lambda x: ' & '.join(set(x)),
    'Details': lambda x: ' ||| '.join(set(x))
}).reset_index()

mkp_filtrado['Order ID'] = mkp_filtrado['Order ID'].astype(str)

mkp = mkp.drop_duplicates(subset='Order ID', keep='first')

#### (YYYY-MM-DD)
mkp['Day']       = mkp['Date created'].dt.strftime('%Y-%m-%d')
mkp['Week Num']   = mkp['Date created'].apply(lambda x: weeknum(x.date()))


#################################
### Sales
sales_bp = sales_bp[sales_bp['Country'] != '']

sales_bp['Sales'] = sales_bp['Sales'].astype(np.int32)
sales_bp['Date'] = pd.to_datetime(sales_bp['Date'])

sales_bp['Week Num'] = sales_bp['Date'].apply(lambda x: weeknum(x.date()))

sales_bp['PK'] = sales_bp['Date'].astype(str).str.cat(sales_bp['Country'].astype(str), sep=' ')

sales_bp = sales_bp.drop_duplicates(['PK'])

#################################
### "Llamadas Caidas"
final_dataframe['Fecha'] = pd.to_datetime(final_dataframe['Fecha'], errors='coerce')
final_dataframe['Agente'] = final_dataframe['Agente'].str.strip()

#################################
### Headcount
hc_data = hc_data[['Agent','Turno','Team Leader','Pais Actual']].rename(columns={
    'Agent': 'Nombre del agente asignado'
    ,'Pais Actual': 'Country'
})

hc_data['Nombre del agente asignado'] = hc_data['Nombre del agente asignado'].str.strip()

#################################
### PhoneIQ
# Cambios de tipo de dato por columnas
iq['Date'] = pd.to_datetime(iq['Date'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
iq = iq.sort_values(by='Date', ascending=True)

iq['Talk Time'] = pd.to_numeric(iq['Talk Time'], errors='coerce').fillna(0).astype(int)
iq['Wait Time'] = pd.to_numeric(iq['Wait Time'], errors='coerce').fillna(0).astype(int)

# Modificaciones con base a la columna 'Date'
iq['Week Num']  =  iq['Date'].apply(lambda x: weeknum(x.date()))
iq['Day']       =  iq['Date'].dt.strftime('%Y-%m-%d')

# Sepacación de la columna 'Queue', para optener los queues y la cola
iq[['Cola', 'Raw Queues']] = iq['Queue'].str.split(': ', expand=True)

# País con base a la cola del 'queues'
iq['País'] = iq['Cola'].map(queues).fillna('Indefinido')

# Calculo del AHT, solo para eventos 'handled'
iq['AHT'] = iq.apply(lambda x: float(x['Talk Time']) / 60 if x['Event'].lower() == 'handled' and pd.notnull(x['Talk Time']) else np.nan, axis=1)

# Calcular el SLA
iq['SLA'] = iq.apply(lambda x: 1 if (x['Event'].lower() == 'handled' and x['Wait Time'] <= 30) else np.nan, axis=1)

# Conteo por tipo de evento
iq['Q'] = iq.apply(lambda x: 1 if x['Event'].lower() == 'handled' else np.nan, axis=1)

#Limpieza de queues
iq['Queue General'] = iq['Raw Queues'].map(colas).fillna(iq['Raw Queues'])

# Cruce con el HC
iq = pd.merge(iq, hc_data[['Nombre del agente asignado', 'Team Leader']], left_on='Agent', right_on='Nombre del agente asignado',how='left')

iq['Agent'] = iq['Agent'].astype(str)
iq['Agent'] = iq['Agent'].str.strip()
iq['Agent'] = iq['Agent'].replace(['Silvia  R TM', 'Zaulo  C TM'],['Silvia R TM', 'Zaulo C TM'])

#################################
### Zendesk Incoming
# Step 2: Clean and preprocess data
vacio['Correo electrónico del solicitante'] = vacio['Correo electrónico del solicitante'].replace(r'\s+', None, regex=True)
vacio['Correo electrónico del solicitante'] = vacio['Correo electrónico del solicitante'].fillna(vacio['Nombre del solicitante'])

# Sort data once
zendesk = vacio.sort_values(by='ID del ticket', ascending=True)

# Step 3: Apply filters and transformations
zendesk = zendesk[zendesk['Rol del solicitante'] == 'End-user']
zendesk = zendesk[~zendesk['Nombre del agente asignado'].isin(UsuariosNOCX)]
zendesk['Correo electrónico del solicitante'] = zendesk['Correo electrónico del solicitante'].astype(str)
zendesk = zendesk[~zendesk['Correo electrónico del solicitante'].str.contains(pattern, na=False)]

# Clean 'Order ID'
zendesk['Order ID'] = zendesk['Order ID'].str.split(r'[y,-,/,]').str[0]
zendesk['Order ID'] = zendesk['Order ID'].astype(str).str.replace(r'[^0-9DO|PA|PY|CO|CR|cr|EC]+', '', regex=True)

# Country cleanup
zendesk['País'] = zendesk['País'].str.strip().fillna('Indefinido')

zendesk['Ticket creado - Marca de tiempo'] = pd.to_datetime(zendesk['Ticket creado - Marca de tiempo'])
zendesk['Ticket creado - Fecha'] = zendesk['Ticket creado - Marca de tiempo'].fillna(zendesk['Ticket creado - Fecha'])
zendesk['Ticket creado - Fecha'] = pd.to_datetime(zendesk['Ticket creado - Fecha'], errors='coerce')
zendesk['Day'] = zendesk['Ticket creado - Fecha'].dt.strftime('%Y-%m-%d')

zendesk.loc[(zendesk['Canal del ticket'] == 'Messaging') & (zendesk['Incoming chat sin fuera de horario'] == False),
       ['Tiempo de primera respuesta (min)', 'Tiempo de resolución completa (min)']] = np.nan

zendesk['Motivo de consulta '] = zendesk['Motivo de consulta '].replace(r'^\s*$', np.nan, regex=True)
#################################
### Incidents

novelties['OID'] = novelties['OID'].astype(str)

novelties['Date'] = pd.to_datetime(novelties['Date'])

novelties = novelties.rename(columns={'OID' : 'Order ID'})

novelties = novelties.groupby(['Order ID','Date']).agg({
    'Incidents': lambda x: ' & '.join(set(x)),
}).reset_index()

###########################
### RRSS Públicas
allagents = allagents.rename(columns={'Categoría de consulta' : 'Motivo de consulta '})

allagents['ID del ticket'] = range(1, len(allagents) + 1)

allagents['Índice de satisfacción del ticket'] = 'Unoffered'

allagents['País'] = allagents['País'].replace('', 'Indefinido')

allagents = allagents[~allagents['Llamada'].astype(str).str.contains('None')]

allagents['Channel'] = 'RRSS'

allagents['Ticket creado - Fecha'] = pd.to_datetime(allagents['Ticket creado - Fecha'], errors='coerce')
allagents['Day'] = allagents['Ticket creado - Fecha'].dt.strftime('%Y-%m-%d')

allagents['País'] = allagents['País'].str.strip().fillna('Indefinido')

productivity_rrss = allagents.pivot_table(
    index=['Day','Nombre del agente asignado','Channel']
    ,values='ID del ticket'
    ,aggfunc='count'
    ,fill_value=0
).reset_index().rename(columns={'ID del ticket':'Resueltos'})

#################################
### Zendesk - Productivity
products = products.rename(columns={'Agente' : 'Nombre del agente asignado','Ticket creado - Fecha':'Day'})

products = pd.concat([products, productivity_rrss], ignore_index=True)

products['Day'] = pd.to_datetime(products['Day'])
products['Week Num']   = products['Day'].apply(lambda x: weeknum(x.date()))
products['Nombre del agente asignado'] = products['Nombre del agente asignado'].str.strip()

#################################
### Zendesk - Productivity | %NO Solved
chat_solved = chat_solved.pivot(
    index=["Agente", "Ticket creado - Fecha"]
    , columns="Etiquetas de ticket"
    , values="tkts"
).reset_index()

chat_solved.columns.name = None  # Remove the multi-index column name

chat_solved = chat_solved.rename_axis(None, axis=1)  # Flatten index

chat_solved['%'] = (100-(chat_solved[['chat_seguimiento_mail', 'cliente_no_responde']].sum(axis=1, skipna=True)))

chat_solved = chat_solved.drop(columns=['chat_seguimiento_mail', 'cliente_no_responde'])

chat_solved = chat_solved.rename(columns={
    'Agente' : 'Nombre del agente asignado'
})

chat_solved['Day'] = pd.to_datetime(chat_solved['Ticket creado - Fecha'])

chat_solved['Channel'] = 'Chat'

chat_solved['Nombre del agente asignado'] = chat_solved['Nombre del agente asignado'].str.strip()

############################

allagents_without_zendesk = allagents[~allagents['Correo electrónico del solicitante'].isin(zendesk['ID del ticket'].astype(str))]

allagents_without_zendesk['Canal del ticket'] = 'RRSS P'

allagents_without_zendesk = allagents_without_zendesk[~allagents_without_zendesk['Correo electrónico del solicitante'].apply(lambda x: re.sub(r'#', '', str(x))).str.contains(r'\d')]

interacciones = pd.concat([zendesk, allagents_without_zendesk], ignore_index=True)

interacciones['Llamada'] = interacciones['Llamada'].astype(str)
interacciones['Canal del ticket'] = interacciones['Canal del ticket'].astype(str)

condiciones = [
    (interacciones['Canal del ticket'] == 'Web')  & (interacciones['Llamada'] == 'True')
    ,(interacciones['Canal del ticket'] == 'Web') & (interacciones['Llamada'] == 'False')
    ,(interacciones['Canal del ticket'] == 'Api')
    ,(interacciones['Canal del ticket'] == 'Messaging')
    ,(interacciones['Canal del ticket'] == 'APPS')
    ,(interacciones['Canal del ticket'] == 'GMB')
    ,(interacciones['Canal del ticket'] == 'TQS')
    ,(interacciones['Canal del ticket'].str.startswith('Twitter'))
    ,(interacciones['Canal del ticket'].str.startswith('Facebook'))
    ,(interacciones['Canal del ticket'].str.startswith('Instagram'))
    ,(interacciones['Canal del ticket'].str.startswith('LinkedIn'))
    ,(interacciones['Canal del ticket'].str.startswith('TikTok'))
    ,(interacciones['Canal del ticket'].str.startswith('App'))
]

interacciones['Channel'] = np.select(condiciones, resultados, default=interacciones['Canal del ticket'])
interacciones['Order ID'] = interacciones['Order ID'].astype(str)
interacciones['Week Num'] = interacciones['Ticket creado - Fecha'].apply(lambda x: weeknum(x.date()) if pd.notna(x) else '')

#####
homologation = catmatrix[['Categoría de consulta', 'Motivo de consulta']]
homologation = homologation.rename(columns={'Motivo de consulta' : 'Homologateds'})
homologation_clean = homologation[homologation['Categoría de consulta'] != ''].drop_duplicates(subset='Categoría de consulta')

####
segments = catmatrix[['Motivo de consulta', 'Segmento']]
segments_clean = segments[(segments['Motivo de consulta'] != '')].drop_duplicates(subset='Motivo de consulta')
segments_clean = segments_clean.rename(columns={'Motivo de consulta' : 'Union', 'Segmento':'General'})

####
departaments = catmatrix[['Motivo de consulta', 'Departamento']]
departaments_clean = departaments[(departaments['Departamento'] != '')].drop_duplicates(subset=['Motivo de consulta'])
deptos = departaments_clean.copy()
deptos = deptos.rename(columns={'Motivo de consulta' : 'Categoría de consulta'})
deptos['Categoría de consulta'] = deptos['Categoría de consulta'].str.split('::').str[-1].str.strip()
departaments_clean = departaments_clean.rename(columns={'Motivo de consulta' : 'Union'})

############## Mergeds
interacciones['Motivo de consulta '] = interacciones['Motivo de consulta '].str.strip()
interacciones_homologated = pd.merge(interacciones,homologation_clean,on='Categoría de consulta',how='left')
interacciones_homologated['Union'] = interacciones_homologated['Motivo de consulta '].combine_first(interacciones_homologated['Homologateds'])

interacciones_segmented   = pd.merge(interacciones_homologated,segments_clean,on='Union',how='left')
interacciones_with_deptos   = pd.merge(interacciones_segmented,departaments_clean,on='Union',how='left')
cruce = pd.merge(interacciones_with_deptos, mkp_filtrado, on='Order ID', how='left')

cruce = cruce.rename(columns={'Categoría de consulta': 'OLD' , 'Union' : 'Categoría de consulta'})

cruce['Categoría de consulta'] = cruce['Categoría de consulta'].str.split('::').str[-1].str.strip()

cruce['Shipping address country'] = cruce['Shipping address country'].fillna('Reseller')
cruce['Categoría de consulta'] = cruce['Categoría de consulta'].fillna('Unknow')
cruce['General'] = cruce['General'].fillna('Unknow')
cruce['Departamento'] = cruce['Departamento'].fillna('Unknow')

cruce['ID del ticket'] = cruce['ID del ticket'].astype(int)
cruce['Tiempo de primera respuesta (min)'] = cruce['Tiempo de primera respuesta (min)'].astype(float)
cruce['Tiempo de primera respuesta (h)'] = cruce['Tiempo de primera respuesta (h)'].astype(float)
cruce['Tiempo de resolución completa (min)'] = cruce['Tiempo de resolución completa (min)'].astype(float)

cruce['País'] = cruce['País'].replace(['', 'Peru'],['Indefinido', 'Perú'])

# Cargas

### DataFrame KPIs

In [ ]:
pivot = cruce[cruce['Day'] >= filter_start_date].drop_duplicates(subset=['ID del ticket']).pivot_table(
    index=['Day','Week Num','País']
    ,columns=['Channel']
    ,values=['ID del ticket','Tiempo de primera respuesta (min)','Tiempo de primera respuesta (h)','Tiempo de resolución completa (min)']
    ,aggfunc={
        'ID del ticket'                         : 'count'
        ,'Tiempo de primera respuesta (min)'    : 'mean'
        ,'Tiempo de primera respuesta (h)'      : 'mean'
        ,'Tiempo de resolución completa (min)'  : 'mean'
        }
    ,margins=True
    ,margins_name='Total'
).reset_index()

pivot.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivot.columns.values]

pivot.insert(0, 'PK', pivot['Day_'].astype(str).str.cat(pivot['País_'].astype(str), sep=' '))

pivot = pivot.drop(columns=[
    'Tiempo de primera respuesta (min)_Email'
    ,'Tiempo de primera respuesta (min)_Phone'
    ,'Tiempo de primera respuesta (min)_RRSS'
    ,'Tiempo de primera respuesta (h)_Chat'
    ,'Tiempo de primera respuesta (h)_Phone'
    ,'Tiempo de primera respuesta (h)_RRSS'
    ,'Tiempo de resolución completa (min)_Email'
    ,'Tiempo de resolución completa (min)_Phone'
    ,'Tiempo de resolución completa (min)_RRSS'
    ])

pivot = pivot.rename(columns={
    'Day_' : 'Day'
    ,'Week Num_' : 'Week Num'
    ,'País_' : 'País'
    ,'ID del ticket_Chat' : 'Chat'
    ,'ID del ticket_Email' : 'Email'
    ,'ID del ticket_Phone' : 'Phone'
    ,'ID del ticket_RRSS' : 'RRSS'
    ,'ID del ticket_RRSS P' : 'RRSS P'
    ,'Tiempo de primera respuesta (h)_Email' : 'FCR Mail (h)'
    ,'Tiempo de primera respuesta (min)_Chat' : 'FCR Chat (min)'
    ,'Tiempo de resolución completa (min)_Chat' : 'Full Resolution Chat (min)'
    })

pivot['Contactos'] = pivot[['Chat', 'Email', 'RRSS', 'RRSS P']].sum(axis=1, skipna=True)

In [ ]:
infocsat = cruce[(cruce['Day'] >= filter_start_date) & (cruce['Índice de satisfacción del ticket'].isin(('Good','Bad')))]

csat2 = infocsat.drop_duplicates(subset=['ID del ticket']).pivot_table(
    index=['Day','Week Num','País'],
    columns=['Channel','Índice de satisfacción del ticket'],
    values='ID del ticket',
    aggfunc='count',
    margins=True,
    margins_name='Calificados'
).reset_index()

csat2.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in csat2.columns.values]

csat2.insert(0, 'PK', csat2['Day_'].astype(str).str.cat(csat2['País_'].astype(str), sep=' '))

csat2 = csat2.drop(columns=['Day_','Week Num_','País_'])

csat2['Goods'] = csat2[['Chat_Good', 'Email_Good', 'Phone_Good', 'RRSS_Good']].sum(axis=1, skipna=True)

In [ ]:
contacts_mkp = cruce[
    (cruce['Day'] >= filter_start_date)
    & (cruce['Shipping address country'] != 'Reseller')
    ].pivot_table(
    index=['Day','Week Num','País']
    ,columns='Shipping address country'
    ,values='ID del ticket'
    ,aggfunc='count'
    ,margins=True
    ,margins_name='MKP Contacts'
).reset_index()

contacts_mkp = contacts_mkp[contacts_mkp['País'] != '']

contacts_mkp.insert(0, 'PK', contacts_mkp['Day'].astype(str).str.cat(contacts_mkp['País'].astype(str), sep=' '))

contacts_mkp = contacts_mkp.rename(columns={
    'CN' : 'Contactos_CN'
    ,'MX' : 'Contactos_MX'
    ,'US' : 'Contactos_US'
    })

contacts_mkp = contacts_mkp.drop(columns=['Day','Week Num','País'])

In [ ]:
calls = iq[iq['Day'] >= filter_start_date].pivot_table(
    index=['Day','Week Num','País']
    ,values=['AHT','SLA','uniqueid','Wait Time','Q']
    ,aggfunc={
        'AHT': 'mean'
        ,'SLA': 'count'
        ,'uniqueid': 'count'
        ,'Wait Time': 'mean'
        ,'Q': 'sum'
        }
).reset_index()

calls.insert(0, 'PK', calls['Day'].astype(str).str.cat(calls['País'].astype(str), sep=' '))

calls = calls.rename(columns={'uniqueid':'Calls','Q':'Handleds'})

In [ ]:
backup = mkp[mkp['Day'] >= filter_start_date].copy()

duplicates = backup.drop_duplicates(subset='Order ID')

mkp_c = duplicates.pivot_table(
    index=['Day','Country MKP']
    ,values='Order ID'
    ,aggfunc='count'
).reset_index()

mkp_c.insert(0, 'PK', mkp_c['Day'].astype(str).str.cat(mkp_c['Country MKP'].astype(str), sep=' '))

mkp_c = mkp_c.rename(columns={'Order ID':'MKP Sales'})

In [ ]:
all = pd.merge(pivot, sales_bp[['PK','Sales']], on='PK', how='left')

all = pd.merge(all, mkp_c[['PK','MKP Sales']], on='PK', how='left')

all = pd.merge(all, calls[['PK','AHT','SLA','Calls','Wait Time','Handleds']], on='PK', how='left')

all = pd.merge(all, csat2, on='PK', how='left')

all = pd.merge(all, contacts_mkp, on='PK', how='left')

all = all[all['País'] != '']


all = all[[
    'PK','Day','Week Num','País'
    ,'Chat','Email','Phone','RRSS','RRSS P','Contactos'
    ,'Sales'
    ,'MKP Sales'
    ,'AHT','SLA','Calls','Wait Time','Handleds'
    ,'Chat_Bad','Chat_Good','Email_Bad','Email_Good','Phone_Bad','Phone_Good','RRSS_Bad','RRSS_Good','Calificados_','Goods'
    ,'FCR Mail (h)'
    ,'FCR Chat (min)'
    ,'Full Resolution Chat (min)'
    ,'Contactos_CN'
    ,'Contactos_MX'
    ,'Contactos_US'
    ,'MKP Contacts'
]]

all['Contactos'] = all[['Contactos', 'Calls']].sum(axis=1, skipna=True)

In [ ]:
Sheet1 = gc.open_by_key('1wYTeulBVwCpQ3HdR-OSHMXfU6X8-7U2QMvxZHTLQFaQ').worksheet('Try1')
Sheet1.clear()
set_with_dataframe(Sheet1, all, include_index=False, resize=True)

### Dataframe Productividad

In [ ]:
dropcalls = final_dataframe.pivot_table(
    index=['Fecha','Agente'],
    values='Motivo',
    aggfunc='count',
    fill_value=0
).reset_index()

dropcalls = dropcalls.rename(columns={
    'Fecha':'Day'
    ,'Agente':'Nombre del agente asignado'
    ,'Motivo' : 'Caídas'
    })

dropcalls['Channel'] = 'Phone'

dropcalls['Day'] = pd.to_datetime(dropcalls['Day'])

In [ ]:
calls_agents = iq[iq['Day'] >= filter_start_date].pivot_table(
    index=['Day','Agent']
    ,values=['AHT','SLA','uniqueid','Wait Time','Q']
    ,aggfunc={
        'AHT': 'mean'
        ,'SLA': 'count'
        ,'uniqueid': 'count'
        ,'Wait Time': 'mean'
        ,'Q': 'sum'
        }
).reset_index()

calls_agents = calls_agents.rename(columns={
    'uniqueid':'Calls'
    ,'Q':'Handleds'
    ,'Agent' : 'Nombre del agente asignado'
    })

calls_agents['Channel'] = 'Phone'

calls_agents['Day'] = pd.to_datetime(calls_agents['Day'])

In [ ]:
prodcutivity = pd.merge(products, hc_data[['Nombre del agente asignado', 'Team Leader','Turno']], on='Nombre del agente asignado',how='left')

prodcutivity = pd.merge(prodcutivity, dropcalls[['Day','Channel','Nombre del agente asignado','Caídas']], on=['Day','Channel','Nombre del agente asignado'],how='left')

prodcutivity = pd.merge(prodcutivity, calls_agents, on=['Day','Nombre del agente asignado','Channel'],how='left')

prodcutivity = pd.merge(prodcutivity, chat_solved, on=['Day','Nombre del agente asignado','Channel'],how='left')

prodcutivity = prodcutivity[prodcutivity['Team Leader'].isin(hc_data['Team Leader'].unique().tolist())]

prodcutivity = prodcutivity.drop(columns=['Ticket creado - Fecha'])

prodcutivity['Day'] = prodcutivity['Day'].dt.strftime('%Y-%m-%d')

prodcutivity = prodcutivity[[
    'Day'
    ,'Nombre del agente asignado'
    ,'Resueltos'
    ,'Bads'
    ,'Goods'
    ,'Respuestas del agente'
    ,'Channel'
    ,'Productividad (min)'
    ,'Tiempo de resolución completa (min)'
    ,'Tiempo de primera respuesta (min)'
    ,'Asignación a primera respuesta (min)'
    ,'Week Num'
    ,'Team Leader'
    ,'Turno'
    ,'Caídas'
    ,'AHT'
    ,'Handleds'
    ,'SLA'
    ,'Wait Time'
    ,'Calls'
    ,'Follow-up'
    ,'%'
]]

,Day,Nombre del agente asignado,Resueltos,Bads,Goods,Respuestas del agente,Channel,Productividad (min),Tiempo de resolución completa (min),Tiempo de primera respuesta (min),...,Team Leader,Turno,Caídas,AHT,Handleds,SLA,Wait Time,Calls,Follow-up,%
1,2024-12-01,Carolay S TM,21.0,0.0,3.0,0.333333,Phone,NaN,NaN,NaN,...,Elohim Simancas,Tarde,2.0,6.538636,22.0,17.0,27.409091,22.0,21.0,NaN
2,2024-12-01,Joice D TM,22.0,1.0,12.0,0.454545,Phone,NaN,NaN,NaN,...,Elohim Simancas,Mañana,2.0,6.459091,22.0,15.0,47.954545,22.0,22.0,NaN
3,2024-12-01,Wendy R TM,11.0,0.0,3.0,0.181818,Phone,NaN,NaN,NaN,...,Elohim Simancas,Tarde,NaN,5.775758,11.0,8.0,34.000000,11.0,11.0,NaN
4,2024-12-01,Zaulo C TM,18.0,0.0,5.0,0.333333,Phone,NaN,NaN,NaN,...,Melissa Novas,Tarde,4.0,6.986508,21.0,16.0,45.666667,21.0,18.0,NaN
6,2024-12-02,Carolay S TM,25.0,0.0,5.0,0.440000,Phone,NaN,NaN,NaN,...,Elohim Simancas,Tarde,4.0,5.826437,29.0,27.0,37.000000,29.0,25.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4156,2025-02-19,Stefany A TM,80.0,NaN,NaN,NaN,RRSS,NaN,NaN,NaN,...,Elohim Simancas,Tarde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4157,2025-02-20,Julieta R TM,62.0,NaN,NaN,NaN,RRSS,NaN,NaN,NaN,...,Melissa Novas,Mañana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4158,2025-02-20,Nathaly G TM,6.0,NaN,NaN,NaN,RRSS,NaN,NaN,NaN,...,Melissa Novas,Mañana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4159,2025-02-20,Stefany A TM,104.0,NaN,NaN,NaN,RRSS,NaN,NaN,NaN,...,Elohim Simancas,Tarde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Sheet_recontacto = gc.open_by_key(gsheet_raw_data)

Sheet_Resumen1 = Sheet_recontacto.worksheet('Raw Data')

Sheet_Resumen1.clear()

set_with_dataframe(Sheet_Resumen1, prodcutivity, include_index=False, resize=True)

## Dataframe ReContacto

In [ ]:
recontacto = cruce

recontacto['Recontacto'] = recontacto.duplicated(subset=['Order ID','Correo electrónico del solicitante', 'Categoría de consulta'], keep='first')

recontacto['Ticket creado - Fecha'] = pd.to_datetime(recontacto['Ticket creado - Fecha'], format='mixed', errors='coerce')

recontacto['Days'] = (
    recontacto['Ticket creado - Fecha'] -
    recontacto.groupby([
        'Order ID', 'Correo electrónico del solicitante', 'País', 'Categoría de consulta', 'Shipping address country'
        ])['Ticket creado - Fecha']
    .transform('min')
).dt.days

recontacto['conteo'] = recontacto.groupby('ID del ticket')['ID del ticket'].transform('count')

recontacto['Tipo'] = recontacto['conteo'].apply(lambda x: 'Tipo 2' if x > 1 else 'Tipo 1')

recontacto = recontacto.drop(columns=['conteo'])

def precompra(row):
    General = row['General']
    Recontacto = row['Recontacto']
    Days = row['Days']

    if General == 'PRE COMPRA (PC)' and Recontacto == True and Days >= 50:
        return 'False'
    elif General == 'Quiero realizar una compra' and Recontacto == True and Days >= 50:
        return 'False'
    else:
      return Recontacto

def sincero(row):
    General = row['General']
    OID = row['Order ID']

    if  General == 'PRE COMPRA (PC)' and OID == '0':
        return ''
    elif  General == 'Quiero realizar una compra' and OID == '0':
        return ''
    else:
      return OID

recontacto['Recontacto'] = recontacto[['General','Recontacto','Days']].apply(precompra, axis=1)
recontacto['Order ID'] = recontacto[['General','Order ID']].apply(sincero, axis=1)

#######################################################

recontacto_filtrado = recontacto[
    (recontacto['Recontacto'] == True)
    & (recontacto['Ticket creado - Fecha'] >= start_date)
    & (recontacto['Ticket creado - Fecha'] <= end_date)
    & (recontacto['Categoría de consulta'] != '')
][['Order ID', 'Correo electrónico del solicitante', 'Categoría de consulta']]

cruce_rc = cruce.merge(recontacto_filtrado, on=['Order ID', 'Correo electrónico del solicitante', 'Categoría de consulta'], how='inner').drop_duplicates(subset=['ID del ticket'])

cruce_rc = cruce_rc[[
    'ID del ticket'
    ,'Categoría de consulta'
    ,'País'
    ,'Ticket creado - Fecha'
    ,'Índice de satisfacción del ticket'
    ,'Channel'
    ,'Correo electrónico del solicitante'
    ,'Nombre del agente asignado'
    ,'Order ID'
    ,'Shipping address country'
    ,'General'
    ,'Recontacto'
    ,'Days'
    ,'Tipo'
]]

cruce_rc = cruce_rc.sort_values(by=['Correo electrónico del solicitante','Ticket creado - Fecha'], ascending=[False,True])

cruce_rc.to_csv(f'{file_name}', index=False)

#######################################################

recontacto = recontacto[(recontacto['Ticket creado - Fecha'] > filter_start_date)]

recontacto = recontacto[[
    'ID del ticket'
    ,'Categoría de consulta'
    ,'País'
    ,'Ticket creado - Fecha'
    ,'Índice de satisfacción del ticket'
    ,'Channel'
    ,'Correo electrónico del solicitante'
    ,'Nombre del agente asignado'
    ,'Order ID'
    ,'Shipping address country'
    ,'General'
    ,'Recontacto'
    ,'Days'
    ,'Tipo'
]]

In [ ]:
Sheet_recontacto = gc.open_by_key('1CMKdx6-B_Wk63HM0dSpeh9sPAOWdJqw81CM2ZGd5l_s')

Sheet_Resumen1 = Sheet_recontacto.worksheet('Sheet1')

Sheet_Resumen1.clear()

set_with_dataframe(Sheet_Resumen1, recontacto, include_index=False, resize=True)

## Dataframe Cat. / motivos de consulta nunique

In [ ]:
ptcatconsulta = cruce[cruce['Day'] >= filter_start_date].drop_duplicates(subset=['ID del ticket']).pivot_table(
    index=['Day', 'País', 'Categoría de consulta','General','Channel','Shipping address country','Departamento']
    ,columns='Índice de satisfacción del ticket'
    ,values=['ID del ticket']
    ,aggfunc={'ID del ticket' : 'count'}
    ,fill_value=0
).reset_index()

ptcatconsulta.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in ptcatconsulta.columns.values]

ptcatconsulta['Tickets'] = ptcatconsulta[['ID del ticket_Bad', 'ID del ticket_Good', 'ID del ticket_Offered', 'ID del ticket_Unoffered']].sum(axis=1, skipna=True)


catconsulta = ptcatconsulta.drop(columns=[
    'ID del ticket_Offered'
    ,'ID del ticket_Unoffered'
    ])

catconsulta = catconsulta.rename(columns={
    'Day_' : 'Date'
    ,'País_' : 'Country'
    ,'Categoría de consulta_' : 'Categoría de consulta'
    ,'General_' : 'General'
    ,'Channel_' : 'Channel'
    ,'Shipping address country_' : 'Shipping country'
    ,'Departamento_' : 'Departamento'
    ,'ID del ticket_Good' : 'Goods'
    ,'ID del ticket_Bad' : 'Bads'
    ,'Shipping address country' : 'Shipping country'
    })


catconsulta = catconsulta[['Date','Country','Categoría de consulta','General','Shipping country','Tickets','Goods','Bads','Channel','Departamento']]

'''
catalogoinfo = cruce[
    (cruce['Shipping address country'] != 'Reseller')
    & (cruce['Day'] >= mkp_date_start)
].to_csv(f'{drivecatalogo}/Historico Contact Rate MKP.csv', index=False)
'''

"\ncatalogoinfo = cruce[\n    (cruce['Shipping address country'] != 'Reseller')\n    & (cruce['Day'] >= mkp_date_start)\n].to_csv(f'{drivecatalogo}/Historico Contact Rate MKP.csv', index=False)\n"

In [ ]:
Sheet_catconsulta = gc.open_by_key(gsheet_cats)

Sheet_Resumen2 = Sheet_catconsulta.worksheet('Sheet1')

Sheet_Resumen2.clear()

# Introduce a delay to respect Google Sheets API rate limits
time.sleep(60) # Wait for 60 seconds

set_with_dataframe(Sheet_Resumen2, catconsulta, include_index=False, resize=True)

## Follow Up - "Tiempos MIAMI"

In [ ]:
time_reasons = {
    'Mi compra no se termina de procesar en el depósito'
    ,'Mi orden ya debería de haber llegado al depósito'
}

raw_data_wh = cruce[
    (cruce['Day'] >= '2025-01-01')
    & (cruce['Categoría de consulta'].isin(time_reasons))
    ].drop_duplicates(subset=['ID del ticket'])

raw_data_wh = raw_data_wh[[
    'ID del ticket'
    ,'Categoría de consulta'
    ,'País'
    ,'Day'
    ,'Índice de satisfacción del ticket'
    ,'Channel'
    ,'Correo electrónico del solicitante'
    ,'Order ID'
    ,'Shipping address country'
]].rename(columns={'Day' : 'Fecha'})

raw_data_wh

#time.sleep(60)

Sheet_follow_up = gc.open_by_key(gsheet_followup)

Sheet_Resumen5 = Sheet_follow_up.worksheet('Raw Contacts')

Sheet_Resumen5.clear()


set_with_dataframe(Sheet_Resumen5, raw_data_wh, include_index=False, resize=True)

# Incidencias

In [ ]:
# Ensure date columns are of datetime type
cruce['Ticket creado - Fecha'] = pd.to_datetime(cruce['Ticket creado - Fecha'])
novelties['Date'] = pd.to_datetime(novelties['Date'])
cruce['Order ID'] = cruce['Order ID'].astype(str)
novelties['Order ID'] = novelties['Order ID'].astype(str)

# Merge DataFrames on 'Order ID' and add suffixes to distinguish columns
merged = pd.merge(cruce, novelties, on='Order ID', how='inner')

categories = [
    'Mi compra no se termina de procesar en el depósito'
    ,'Mi orden ya debería de haber llegado al depósito'
    ,'No me sirve el tiempo de entrega'
    ,'Cómo cancelar mi compra'
    ,'Me notificaron de un inconveniente'
    ,'Quiero modificar los productos de mi compra'
    ,'Quiero cambiar datos en una orden'
]

# Filter based on the conditions
maches = merged[
    (merged['Ticket creado - Fecha'] >= merged['Date'])
    & (merged['Ticket creado - Fecha'] <= merged['Date'] + pd.Timedelta(days=6))
    & (merged['Categoría de consulta'].isin(categories))
][['Order ID', 'Ticket creado - Fecha', 'ID del ticket', 'Date', 'Incidents']]

sends = novelties.pivot_table(
    index='Incidents'
    ,values='Order ID'
    ,aggfunc='nunique'
    ,fill_value=0
).reset_index()

sends = sends.rename(columns={'Order ID':'Comunicados Enviados'})


count = maches.pivot_table(
    index=['Incidents',]
    ,values=['Order ID','ID del ticket']
    ,aggfunc={'Order ID' : 'nunique','ID del ticket':'count'}
    ,fill_value=0
).reset_index()

count = pd.merge(sends, count, on='Incidents', how='left')

count = count[['Incidents','Comunicados Enviados','Order ID','ID del ticket']].rename(columns={
    'Incidents':'Comunicado'
    ,'Order ID':'Clientes Contactaron'
    ,'ID del ticket':'Tickets Creados'})

count['% ReContacto'] = count['Clientes Contactaron']/count['Comunicados Enviados']

count = count.sort_values(by='Comunicado', ascending=False)

Sheet_novelties = gc.open_by_key(gsheet_incidents)

Sheet_Resumen4 = Sheet_novelties.worksheet('Impacto WISMO')

Sheet_Resumen4.clear()

set_with_dataframe(Sheet_Resumen4, count, include_index=False, resize=True)